In [1]:
import pandas as pd
import csv
import requests
import numpy as np
import json
import plotly.express as px
import plotly.io as pio


#Download_csv_file

CSV_URL = "https://api.covid19tracker.in/data/csv/latest/states.csv"
with requests.Session() as s:
    download = s.get(CSV_URL)


#Read and write to csv

with open('covid19india_states.csv', 'w') as f:
    writer = csv.writer(f)
    for line in download.iter_lines():
        writer.writerow(line.decode('utf-8').split(','))


#Creating Pivot table

districts=pd.read_csv('covid19india_states.csv')
table = pd.pivot_table(districts, values='Confirmed', index=['state'], columns=['Date'], fill_value=0)
table.to_csv("state_pivot.tsv", sep='\t')
pivot_table=pd.read_csv("state_pivot.tsv", sep="\t")

#Creating two weeks data

two_weeks = pivot_table.T.tail(14).T
first_week = two_weeks.T.head(7).T
last_week = two_weeks.T.tail(7).T
first_column = pivot_table.T.head(2).T


first_week['avg_cases_previous_week'] = first_week.mean(axis=1)
last_week['avg_cases_over_7_days'] = last_week.mean(axis=1)
week1 = first_column.join(first_week)
week2 = first_column.join(last_week)


merged = week1.merge(week2, on=['state'])
merged['Increase'] = merged['avg_cases_over_7_days']-merged['avg_cases_previous_week']
merged['Rate_of_Increase']=(merged['Increase']/merged['avg_cases_previous_week'])*100


searchfor=['India','state Unassigned','Unknown']
merged2 = merged[~merged.state.str.contains('|'.join(searchfor))]


header=['state','avg_cases_over_7_days','avg_cases_previous_week','Increase','Rate_of_Increase']
merged2.to_csv("state.tsv", sep='\t',columns=header)


#Creating image\n",
pio.renderers.default = 'browser'

india_states = json.load(open("india_states.geojson", "r"))

state_id_map = {}
for feature in india_states["features"]:
    feature["id"] = feature["properties"]["st_code"]
    state_id_map[feature["properties"]["id"]] = feature["id"]


df = pd.read_csv("state.tsv",sep='\t')
df["id"] = df["state"].apply(lambda x: state_id_map[x])
df.loc[df['Rate_of_Increase'] <= 0, 'Increase_image'] = 0
df.loc[df['Rate_of_Increase'] > 0, 'Increase_image'] = df['Rate_of_Increase']

fig = px.choropleth(
    df,
    locations="id",
    geojson=india_states,
    color="Rate_of_Increase",
    hover_name="state",
    hover_data=["Rate_of_Increase"],
    title="Rate of Increase in COVID19 cases per week / Date analyzed: 15-11-2021 ",
    color_continuous_scale=px.colors.diverging.BrBG,
    color_continuous_midpoint=0.1)
fig.update_layout(coloraxis_colorbar=dict(
    title="Rate of Increase"))
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [2]:
df.sort_values(by=['Rate_of_Increase'], inplace=True, ascending=False, ignore_index=True)
df.loc[0:9, ['state', 'Rate_of_Increase']]

,state,Rate_of_Increase
0,Mizoram,2.470745
1,Kerala,0.752508
2,Ladakh,0.522009
3,Himachal Pradesh,0.361650
4,West Bengal,0.331130
5,Jammu and Kashmir,0.303700
6,Manipur,0.285033
7,Assam,0.260062
8,Nagaland,0.215366
9,Tamil Nadu,0.204784
